In [1]:
import sys

sys.path.insert(0, '..')

## Setup the model area

In [2]:
from morpheus.project.types.discretization.spatial import ActiveCells
from morpheus.project.types.discretization import SpatialDiscretization
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit, Grid, Crs
from morpheus.project.types.geometry import Polygon, LineString

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(0)
length_unit = LengthUnit.meters()
relative_col_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
relative_row_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  relative_col_coordinates=relative_col_coordinates,
  relative_row_coordinates=relative_row_coordinates,
  rotation=rotation,
)

spatial_discretization = SpatialDiscretization(
  geometry=polygon,
  grid=grid,
  affected_cells=ActiveCells.from_polygon(polygon=polygon, grid=grid),
  crs=Crs.from_str('EPSG:4326')
)

spatial_discretization



SpatialDiscretization(geometry=Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96629040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon'), grid=Grid(origin=Point(coordinates=(13.922224410263878, 50.9662904037036), type='Point'), col_widths=[33.68941547288559, 33.68941547288559, 33.6894154728856, 33.689415472885585, 33.6894154728856, 33.6894154728856, 33.68941547288557, 33.6894154728856, 33.68941547288563, 33.68941547288557], total_width=336.89415472885594, row_heights=[27.74944856585935, 27.74944856585935, 27.749448565859346, 27.749448565859353, 27.74944856585934, 27.749448565859353, 27.749448565859325, 27.74944856585938, 27.749448565859325, 27.749448565859353], total_height=277.4944856585935, rotation=Rotation(value=0), length_unit=LengthUnit(unit=2)), affected_cells=ActiveCells(shape=(10, 10), data=[ActiveCell(col=0, row=0), ActiveCell(col=0, row

## Setup Time Discretization

In [3]:
from morpheus.project.types.discretization.time.TimeUnit import TimeUnit
from morpheus.project.types.discretization.time.Stressperiods import StartDateTime, StressPeriodCollection, \
  StressPeriod, NumberOfTimeSteps, TimeStepMultiplier, IsSteadyState
from morpheus.project.types.discretization.time import TimeDiscretization
from datetime import datetime

time_discretization = TimeDiscretization(
  start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
  end_date_time=StartDateTime.from_datetime(datetime(2020, 12, 31)),
  stress_periods=StressPeriodCollection([
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.yes()
    ),
  ]),
  time_unit=TimeUnit.days()
)

time_discretization

TimeDiscretization(start_date_time=StartDateTime(value=datetime.datetime(2020, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)), end_date_time=StartDateTime(value=datetime.datetime(2020, 12, 31, 0, 0, tzinfo=datetime.timezone.utc)), stress_periods=StressPeriodCollection(values=[StressPeriod(start_date_time=StartDateTime(value=datetime.datetime(2020, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)), number_of_time_steps=NumberOfTimeSteps(value=1), time_step_multiplier=TimeStepMultiplier(value=1), steady_state=IsSteadyState(value=True))]), time_unit=TimeUnit(unit=4))

# Setup Model

In [4]:
from morpheus.project.types.Model import Model

model = Model.new()
model = model.with_updated_time_discretization(time_discretization)
model = model.with_updated_spatial_discretization(spatial_discretization=spatial_discretization)

In [5]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))

m.fit_bounds(m.get_bounds())
m

# Setup constant head boundary with 1 observation

In [6]:
from morpheus.project.types.boundaries.Observation import ObservationId, ObservationName
from morpheus.project.types.geometry import Point
from morpheus.project.types.boundaries.Boundary import BoundaryName, ConstantHeadBoundary
from morpheus.project.types.layers import LayerId
from morpheus.project.types.boundaries.ConstantHeadObservation import ConstantHeadObservation, ConstantHeadRawDataItem, \
  HeadValue

layers_list = [LayerId.new()]
constant_head = ConstantHeadBoundary.from_geometry(
  name=BoundaryName('constant_head'),
  geometry=LineString([
    (13.9223, 50.9662),
    (13.9250, 50.96615),
  ]),
  grid=grid,
  affected_layers=[model.layers[0].layer_id],
  observations=[
    ConstantHeadObservation(
      observation_id=ObservationId.new(),
      observation_name=ObservationName.default(),
      geometry=Point((13.9223, 50.9662)),
      data=[
        ConstantHeadRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2019, 1, 1)),
          head=HeadValue.from_float(100),
        ),
        ConstantHeadRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2021, 12, 31)),
          head=HeadValue.from_float(100),
        ),
      ]
    )
  ]
)

m.add_child(folium.GeoJson(constant_head.as_geojson()))
m.add_child(folium.GeoJson(constant_head.affected_cells.to_geojson(grid).as_geojson()))
for constant_head.observation in constant_head.observations:
  m.add_child(folium.GeoJson(constant_head.observation.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [7]:
from morpheus.project.types.boundaries.Boundary import BoundaryCollection

boundaries = BoundaryCollection.new()
boundaries.add_boundary(constant_head)
model = model.with_updated_boundaries(boundaries=boundaries)

## Calculate the model asynchronously with celery

In [8]:

from morpheus.project.infrastructure.calculation.engines.modflow_2005.types.Mf2005CalculationEngineSettings import \
  Mf2005CalculationEngineSettings

calculation_engine_setting = Mf2005CalculationEngineSettings.default()
calculation_engine_setting

Mf2005CalculationEngineSettings(packages=PackageData(oc=OcPackageData(ihedfm=0, iddnfm=0, chedfm=None, cddnfm=None, cboufm=None, compact=True, stress_period_data=None, extension=['oc', 'hds', 'ddn', 'cbc', 'ibo'], unitnumber=None, filenames=None, label='OC'), flow_package=FlowPackage(type='lpf'), solver_package=SolverPackageData(type='pcg', data=PcgPackageData(mxiter=50, iter1=30, npcond=1, hclose=1e-05, rclose=1e-05, relax=1.0, nbpol=0, iprpcg=0, mutpcg=3, damp=1.0, dampt=1.0, ihcofadd=0, extension='pcg', unitnumber=None, filenames=None))))

In [9]:
from morpheus.project.infrastructure.persistence.CalculationRepository import calculation_repository
from morpheus.project.types.calculation.CalculationProfile import CalculationProfile, CalculationEngineType
from morpheus.project.types.calculation.Calculation import Calculation

calculation = Calculation.new(
  model=model,
  calculation_profile=CalculationProfile.new(CalculationEngineType.MF2005)
)
calculation_repository.save_calculation(calculation)
calculation.calculation_id

CalculationId(value='191c19bc-4087-4e69-b43a-8ac9d7737d1f')

In [10]:
from morpheus.project.tasks import run_calculation_by_id

result = run_calculation_by_id.delay(calculation.calculation_id.to_str())
result_state = result.state
result_id = result.id
{'id': result_id, 'state': result_state}

{'id': '523cc463-d98b-4d58-b98d-38156dd52864', 'state': 'PENDING'}

In [11]:
from celery.result import AsyncResult

result = AsyncResult(id=result_id)
{
  'id': result.id,
  'state': result.state,
  'ready': result.ready(),
  'successful': result.successful(),
  'value': result.result if result.ready() else None,
}

{'id': '523cc463-d98b-4d58-b98d-38156dd52864',
 'state': 'PENDING',
 'ready': False,
 'successful': False,
 'value': None}

In [12]:
calculation = calculation_repository.get_calculation(calculation.calculation_id)
calculation

Calculation(calculation_id=CalculationId(value='191c19bc-4087-4e69-b43a-8ac9d7737d1f'), model=Model(model_id=ModelId(value='72bad54b-a3a9-47f0-a822-c4d48a2fa552'), spatial_discretization=SpatialDiscretization(geometry=Polygon(coordinates=[[[13.922514437551428, 50.964720483303836], [13.925250781947113, 50.965228748412386], [13.925036413951403, 50.96623732041704], [13.92222441026388, 50.96629040370362], [13.922514437551428, 50.964720483303836]]], type='Polygon'), grid=Grid(origin=Point(coordinates=[13.922224410263878, 50.9662904037036], type='Point'), col_widths=[33.68941547288559, 33.68941547288559, 33.6894154728856, 33.689415472885585, 33.6894154728856, 33.6894154728856, 33.68941547288557, 33.6894154728856, 33.68941547288563, 33.68941547288557], total_width=336.89415472885594, row_heights=[27.74944856585935, 27.74944856585935, 27.749448565859346, 27.749448565859353, 27.74944856585934, 27.749448565859353, 27.749448565859325, 27.74944856585938, 27.749448565859325, 27.749448565859353], to

In [13]:
calculation.calculation_state

<CalculationState.CREATED: 'created'>

In [14]:
calculation.calculation_log

In [15]:
calculation.calculation_result